[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/seap-udea/tspice/blob/main/examples/love_numbers_quickstart.ipynb)

# Planetary Tidal Response using `tSPICE`: Quickstart

This notebook demonstrates the calculation of Love numbers for a planetary body using `tSPICE`.

In [2]:
try:
    from google.colab import drive
    %pip install -Uq tspice
except ImportError:
    print("Not running in Colab, skipping installation")
    %load_ext autoreload
    %autoreload 2

# Uncomment to install from GitHub (development version)
# !pip install git+https://github.com/seap-udea/tspice.git

Not running in Colab, skipping installation
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Import package and initialize it (load data required for computations)

In [8]:
#Import TSPICE and initialize the kernels
import tspice as tsp
tsp.initialize(verbose=False)

TSPICE initialized successfully. Kernels loaded from: /Users/jzuluaga/dev/tspice/examples/tspice_data/meta_kernel


## Responding body

We start by defining our responding body:

In [13]:
earth_interior = tsp.BodyResponse('Earth')
earth_interior.scale_constants(verbose=True)

Characteristic scales for adimensionalization:
Length scale L = 6.37e+06 m
Mass scale M = 5.97e+24 kg
Density scale RHO = 2.31e+04 kg/m^3
Pressure/Elasticity scale P = 1.44e+12 Pa
Velocity scale V = 7.91e+03 m/s
Time scale T = 8.05e+02 s
Angular frequency scale OMEGA = 1.24e-03 rad/s
Gravity scale Gad = 9.82e+00 m/s^2


## Planetary profile

The profile we use to test the Love Numbers calculation is a modification of the PREM suggested by [Amorín & Gudkova (2025)](https://doi.org/10.1016/j.pepi.2024.107304). The integration methodology also follows the method outlined in their previous paper [(Amorín & Gudkova, 2024)](https://doi.org/10.1016/j.pepi.2024.107144).

In [14]:
from tspice.planet import Earth
earth_model = Earth()
planet_profile = earth_model.planet_profile

## Response integration

Now we add layers:

In [15]:
layers_list = [dict(name='Outer Core', type='fluid', r0=1221500.0, rf=3480000.0),
               dict(name='Inner Core', type='solid', r0=0, rf=1221500.0),
               dict(name='Mantle + crust', type='solid', r0=3480000.0, rf = earth_interior.L)]

Choose mode:

In [ ]:
f_day = 1.93502  # M2 tide in cycles/day
n = 2

Set integration parameters:

In [16]:
r0_ini = 6e3
earth_interior.set_integration_parameters_ad(n=n, f_days=f_day, layers_list=layers_list, planet_profile=planet_profile, r0_ini=r0_ini)

Integrate:

In [17]:
earth_interior.integrate_internal_solutions_ad(verbose=True)

Integrated first solid layer!
Integrated second fluid layer!
Integrated third solid layer!
Combined all solutions!
Love numbers:
h_2 = 0.609316
l_2 = 0.085643
k_2 = 0.299043
delta_2 = 1.160751


Love numbers are now attributes of the planet:

In [19]:
earth_interior.k_n, earth_interior.h_n, earth_interior.l_n

(np.float64(0.2990433851603629),
 np.float64(0.6093161139942496),
 np.float64(0.08564339441970326))

---
**Copyright**: 2026, Deivy Mercado, Jorge I. Zuluaga, Gloria Moncayo